In [2]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
import datetime as dt
import os
pd.set_option('display.max_columns', None) 

In [3]:
#enter search values into url to filter results
url = 'https://milwaukee.craigslist.org/search/eby/apa?bedrooms=1&pets_dogs=1'

In [4]:
open_site = requests.get(url)

In [5]:
parse_text_1 = bs(open_site.text, 'html.parser').prettify()
parse_text_2 = bs(open_site.text, 'html.parser')

In [6]:
listings = parse_text_2.find_all("p","result-info")
listings[0]

<p class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">favorite this post</span>
</span>
<time class="result-date" datetime="2019-08-10 17:04" title="Sat 10 Aug 05:04:47 PM">Aug 10</time>
<a class="result-title hdrlnk" data-id="6953998743" href="https://milwaukee.craigslist.org/apa/d/grafton-3-bedroom-townhome-available/6953998743.html">3 BEDROOM TOWNHOME AVAILABLE</a>
<span class="result-meta">
<span class="result-price">$1820</span>
<span class="housing">
                    3br -
                    1642ft<sup>2</sup> -
                </span>
<span class="result-hood"> (Grafton)</span>
<span class="result-tags">
<span class="pictag">pic</span>
</span>
<span class="banish icon icon-trash" role="button">
<span class="screen-reader-text">hide this posting</span>
</span>
<span aria-hidden="true" class="unbanish icon icon-trash red" role="button"></span>
<a class="restore-link" href="#">
<span class="restore-narrow-text">restore</span>
<span

In [7]:
n=1
df = pd.DataFrame()
for i in listings:
    index='listing #'+str(n)
    try: 
        description = i.find('a','result-title').contents
    except:
        description = 'Not Available'
    try:
        price = i.find('span','result-price').contents
    except:
        price = 'Not Available'
    try:
        location = i.find('span','result-hood').contents
    except:
        location = 'Not Available'
    try:
        details = i.find('span','housing').contents[0].replace(' ','').replace('\n','').replace('-','')
        details_split = details.split('br')
        br = details_split[0]
        try:
            sq_ft = details_split[1]
        except:
            sq_ft = 'Not Available'
    except:
        details = 'Not Available'
    try:
        anchor_link = i.find('a','result-title hdrlnk',href=True)
        link = anchor_link['href']
    except:
        link='No Link Available'
    full_list = [location, price, br, sq_ft,description,link]
    full_list = pd.Series(full_list)
    df = df.append(full_list, ignore_index=True)
    n=n+1

In [8]:
df = df.rename(columns={0:'location',1:'rent $',2:'bedrooms',3:'sq_ft',4:'description',5:'link'})
df = df.astype(str)
df['location'] = df['location'].str.replace('[','').str.replace(']','').str.replace('(','').str.replace(')','').str.replace("'",'').str.strip().str.capitalize()
df['rent $'] = df['rent $'].str.replace('[','').str.replace(']','').str.replace('$','').str.replace("'",'').str.strip()
df['rent $'] = df['rent $'].astype(int)
df['bedrooms'] = df['bedrooms'].astype(int)
df['sq_ft'] = df['sq_ft'].str.replace('ft','').str.strip()
df['description'] = df['description'].str.replace('[','').str.replace(']','').str.replace('$','').str.replace("'",'').str.strip()

In [9]:
df = df.sort_values(by='rent $', ascending=True)
df_filtered = df[df['location']!='Not available'].reset_index(drop=True)

In [10]:
df_filtered

,location,rent $,bedrooms,sq_ft,description,link
0,Milw,600,1,800,Beautiful Big One Bedroom 55 & Over,https://milwaukee.craigslist.org/apa/d/milwauk...
1,Milwaukee,650,3,2100,Rent to Own Your First Home! 3 Bedroom 2 Bathroom,https://milwaukee.craigslist.org/apa/d/milwauk...
2,Hartford,675,2,,Hartford Apartment,https://milwaukee.craigslist.org/apa/d/hartfor...
3,Mke east_se riverwest-humboldt blvd@center st,699,1,550,MidCenturyModern Courtyard Unit*coveted SE Riv...,https://milwaukee.craigslist.org/apa/d/milwauk...
4,"Mke east near uwm,riverwesthumboldt-center",699,1,500,circa 57-Riverwest Humboldt@Center:On UWM shut...,https://milwaukee.craigslist.org/apa/d/milwauk...
5,2774 s. 60th st. milwaukee,725,2,,2 (2 br) bedroom lower - avail 10/1 Large 5 ro...,https://milwaukee.craigslist.org/apa/d/2-2-br-...
6,311 n 95th st,750,2,,One or Two bed Available in GREAT Location nea...,https://milwaukee.craigslist.org/apa/d/milwauk...
7,311 n 95th st,750,2,,"Two Bed Near MCW, Mayfair Mall, Zoo, EVERYTHING!",https://milwaukee.craigslist.org/apa/d/milwauk...
8,Milwaukee,750,2,,2BR Upper w/ Great Location Close to Grocery S...,https://milwaukee.craigslist.org/apa/d/milwauk...
9,1201 lambeth road,780,2,825,Waukesha . . Lambeth Road . . 4-Fa...,https://milwaukee.craigslist.org/apa/d/waukesh...


In [11]:
date = dt.datetime.now()
month = date.month
month = str(month)
day = date.day
day = str(day)
year = date.year
year = str(year)
hour = date.hour
hour = str(hour)
minute = date.minute
minute = str(minute)
pwd = os.getcwd()
xl_title = pwd+'/Apt List as of '+month+'-'+day+'-'+year+' '+hour+':'+minute+'.xlsx'
xl_title = str(xl_title)

In [12]:
df_filtered.to_excel(xl_title, engine='xlsxwriter')